# MSQ: Necessary Structures

This notebook outlines and creates the necessary code structures to implement the MSQ.

NOTE: this code is written in Python 2.7.x, but all attempts are made to use Python3-compliant syntax. 

In [1]:
# Import relevant libraries
from __future__ import division, print_function
import numpy as np

The following is a very compact description of the method. See the authors' paper for a much more in-depth discussion. 

## Main Method Objects

The key idea is that we define two objects: 

- a set of *quantiles* of some distribution, denote this $\mathbf{q}$, of size $s$;
- a set of *functions of quantiles*, which map a set of quantiles to a vector of reals. denote this $\Phi$. 

This skips a lot of details. For example:

- the above set of quantiles and function of quantiles should be thought of as being applied to a *vector* of random variables of length $J$. 
- For each $j$ random variable, the functions of quantiles should produce a vector of length $M$. Thus the total number of elements in the final vector of reals will be $JM$.

That said, I will use the minimal notation needed to describe the process, and use an example to explore and illustate the details of the process. 

Assume we have two things: 

1. A set of M vectors of emperical realizations of the DGP we are trying to fit -- i.e. empirical distributions drawn from the true unknown DGP
    - call these the "empirical" values
    - denote empirical quantiles of these data $\hat{\mathbf{q}}$
    - denote empirical functions of these quantiles $\hat{\Phi}_j$
2. A parameterized, simulation-based DGP, from which we can simulate draws conditional on a parameter $\theta$, 
    - call these the "theoretical" values
    - denote theoretical quantiles of these data $\mathbf{q}_{\theta}$
    - denote theoretical functions of these quantiles $\Phi_{\theta, j}$


We will explore this in more detail in the example below. 

To fit the theoretical model to the empirical data, choose the parameter $\theta$ to minimize the following quadratic objective function:

$$
\hat{\theta} = \underset{\theta \in \Theta}{\textrm{argmin}} \; \left(\hat{\mathbf{\Phi}} -  \mathbf{\Phi}_{\theta}\right)^{\textrm{T}} \mathbf{W}_{\theta} \left(\hat{\mathbf{\Phi}} -  \mathbf{\Phi}_{\theta}\right).
$$

Here $\mathbf{W}_{\theta}$ is a symmetric positive definite weighting matrix.

In addition, the bolded $\Phi$ values are defined as the "stacked" vectors, for each of the $J$ random variables:

$$
\mathbf{\Phi} = \left(\Phi_{1}^{\textrm{T}}, \Phi_{2}^{\textrm{T}}, ..., \Phi_{j}^{\textrm{T}}, ..., \Phi_{J}^{\textrm{T}} \right)^{\textrm{T}}
$$


for both $\hat{\Phi}$ and $\Phi_{\theta}$.

## Illustration By Example

We'll use the example of the $\alpha$-stable distribution to demonstrate the estimation method.

As discussed in the notebook "An Aside on Alpha Stable Distributions," the $\alpha$-stable distribution is a four-parameter distribution denoted $S(\alpha, \beta, \mu, \sigma)$ where:

- $\alpha \in (0,2]$, is the "tail index:" it measures the thickness of tails of the distribution. 
- $\beta \in [-1,1]$ is the "skewness" parameter. 
- $\sigma \in \mathbb{R}^{+}$ is the "scale" or "dispersion" parameter
- $\mu \in \mathbb{R}$ is the location parameter

There are three special cases of the family of stable distributions:

- Normal: $S(\alpha=2, \beta=NA, \frac{\sigma}{\sqrt{2}}, \mu) \rightarrow \mathscr{N}(\mu, \sigma^2)$
- Cauchy: $S(\alpha=1, \beta=0, \sigma, \mu)$
- Levy: $S(\alpha=0.5, \beta=1, \sigma, \mu)$


Importantly, the $\alpha$ parameter governs whether moments of the distribution exist. For $X \sim S(\alpha, \beta, \mu, \sigma)$:

$$\mathbb{E} \left[ X^{p}\right] \lt \infty \; \forall p \lt \alpha .$$


We use a quantiles-based estimator defined by McCulloch (1986). 

Let's define the two functions of parameters: 

* The theoretical function of parameters, $\Phi_{\theta}$: 

$$
\Phi_{\theta}=\left(
    \begin{array}{c}
        \frac{ q_{0.95, \theta} \,-\, q_{0.05, \theta} } { q_{0.75, \theta} \,-\, q_{0.25, \theta} } \\
        \frac{ (q_{0.95, \theta} \,-\, q_{0.5, \theta}) \,+\, (q_{0.05, \theta} \,-\, q_{0.5, \theta}) } { q_{0.95, \theta} \,-\, q_{0.05, \theta} }  \\
         (q_{0.75, \theta} \,-\, q_{0.25, \theta}) \sigma \\
         q_{0.5, \theta} \sigma + \mu
    \end{array}
    \right)
$$


* The empirical function of parameters, $\hat{\Phi}$: 

$$
\hat{\Phi}=\left(
    \begin{array}{c}
        \frac{ \hat{q}_{0.95} \,-\, \hat{q}_{0.05} } { \hat{q}_{0.75} \,-\, \hat{q}_{0.25} } \\
        \frac{ (\hat{q}_{0.95} \,-\, \hat{q}_{0.5}) \,+\, (\hat{q}_{0.05} \,-\, \hat{q}_{0.5}) } { \hat{q}_{0.95} \,-\, \hat{q}_{0.05} }  \\
         (\hat{q}_{0.75} \,-\, \hat{q}_{0.25}) \\
         \hat{q}_{0.5}
    \end{array}
    \right)
$$


Dominicy and Verdas (2013) follow McCulloch (1986) and standardize the empirical data which comprises $\hat{\Phi}$, and thus additionally standardize the theoretical measurements. 

The theoretical function of parameters $\Phi_{\theta}$ is obtained by drawing a simulated sample of data from the distribution implied by $\theta$, which we will denote as $\overset{\sim}{\Phi}_{\theta}$.

Finally, to represent $\Phi_{\theta}$ we draw $R$ simulation paths and find the average value: 

$$
\overset{\sim}{\Phi}_{\theta}^{R} = \frac{1}{R}\sum_{r=1}^{R} \overset{\sim}{\Phi}_{\theta}^{r}.
$$

We find the $\hat{\theta}$ by minimizing the quadratic objective: 

$$
\hat{\theta} = \underset{\theta \in \Theta}{\textrm{argmin}} \; \left(\hat{\mathbf{\Phi}} - \mathbf{\overset{\sim}{\Phi}}_{\theta}^{R} \right)^{\textrm{T}} \mathbf{W}_{\theta} \left(\hat{\mathbf{\Phi}} -  \mathbf{\overset{\sim}{\Phi}}_{\theta}^{R}\right).
$$



# Appendix A: Next Extension

Extensions:

- Weighted empirical data.